### Generate ensembles from equilibrated state

From the initial conditions, add randomness and run model forward using 10 ensemble members. When do members diverge and become independent?

TODO:

- Wrap method 1 in a function. 

- Create a script. Single ensemble run. Input is what ensemble member are you?

- Slurm can run script in batch. "Job Array" sbatch --array=1-30

- os.environ() pull environment variable into notebook. Use environment variable to pass information aout what em should be running

- environment variables

- apply small purturbation to a single point

- Instead of plotting KE, plot the correlation between the PV fields at each time step. It should fall off.

- module load gcc. pip install -e . pyqg. Use your own developmental repo.

In [2]:
import numpy as np
import xarray as xr
import pyqg
from pyqg import diagnostic_tools as tools
from pyqg import qg_model, particles
import matplotlib.pyplot as plt
from numpy.random import Generator, MT19937, SeedSequence, seed
import os

In [99]:
# parameter for randomness
n = os.getenv('SLURM_STEP_ID')
int(n)

0

#### Import initial conditions from equilibrium run

In [3]:
ds_initial = xr.open_dataset('/burg/abernathey/users/hillary/QG_equilibrium_proto.nc')
initial_PV = ds_initial.q.values

#### Ensemble Generator

In [19]:
# config
year = 24*60*60*360.
day = 24*60*60.
tmax = 2*year
twrite = day
tavestart = day
sig = 1.e-6
n=1
    
def ensemble_generator(initial_PV, n):
    
    # configure model
    m = pyqg.QGModel(tmax=tmax, twrite=twrite, tavestart=tavestart)

    # create an empty array filled with zeros 
    noise = np.zeros_like(ds_initial.q.values)

    # index to the middle of noise and add random perturbation to both levels
    rg = Generator(MT19937(int(n)))
    noise[:,np.floor(len(ds_initial.x)/2).astype('int'), np.floor(len(ds_initial.y)/2).astype('int')] = sig*rg.random((m.q.shape[0]))

    # set PV anomaly with randomness
    m.set_q(ds_initial.q.values + noise)
    
    # run with snapshots, save model increments as xarray DataSet
    datasets = []
    for snapshot in m.run_with_snapshots(tsnapstart=m.t, tsnapint=m.dt):
        model = m.to_dataset()
        model = model.assign(q=(('time','lev','y','x'), m.q.copy()[np.newaxis,...]))
        datasets.append(model)

    ds = xr.concat(datasets, dim='time')
    ds = ds.expand_dims(dim='ensemble_member')
    ds['ensemble_member'] = [n]
    
    return ds
    # Save model to netCDF
#     fn = '/burg/abernathey/users/hillary/QG_proto_EM_'+ str('%03d'%int(n)) +'.nc'
#     ds.to_netcdf(fn, engine='h5netcdf', invalid_netcdf=True)
    
    

In [20]:
ds = ensemble_generator(initial_PV, n)

INFO:  Logger initialized


In [21]:
ds

<xarray.Dataset>
Dimensions:          (ensemble_member: 1, k: 33, l: 64, lev: 2, time: 8640, x: 64, y: 64)
Coordinates: (12/27)
  * time             (time) float64 7.2e+03 1.44e+04 ... 6.22e+07 6.221e+07
  * lev              (lev) int64 1 2
  * x                (x) float64 7.812e+03 2.344e+04 ... 9.766e+05 9.922e+05
  * y                (y) float64 7.812e+03 2.344e+04 ... 9.766e+05 9.922e+05
  * l                (l) float64 0.0 6.283e-06 ... -1.257e-05 -6.283e-06
  * k                (k) float64 0.0 6.283e-06 1.257e-05 ... 0.0001948 0.0002011
    ...               ...
    taveint          float64 8.64e+04
    tsnapint         float64 8.64e+04
    ntd              int64 1
    pmodes           int64 1
    radii            int64 1
  * ensemble_member  (ensemble_member) int64 1
Data variables: (12/20)
    q                (ensemble_member, time, lev, y, x) float64 4.176e-06 ......
    u                (ensemble_member, time, lev, y, x) float64 -0.1212 ... -...
    v                (ensemble_member, time, lev, y, x) float64 -0.06289 ... ...
    ufull            (ensemble_member, time, lev, y, x) float64 0.05753 ... -...
    vfull            (ensemble_member, time, lev, y, x) float64 -0.06289 ... ...
    qh               (ensemble_member, time, lev, l, k) complex128 (0.0020495...
    ...               ...
    EKE              (ensemble_member, time, lev) float64 nan nan ... 6.193e-05
    EKEdiss          (ensemble_member, time) float64 nan nan nan ... nan nan nan
    Ensspec          (ensemble_member, time, lev, l, k) float64 nan ... 2.761...
    KEflux           (ensemble_member, time, l, k) float64 nan nan ... 1.668e-24
    KEspec           (ensemble_member, time, lev, l, k) float64 nan ... 2.001...
    entspec          (ensemble_member, time, l, k) float64 nan nan ... 9.926e-44
Attributes:
    title:       pyqg: Python Quasigeostrophic Model
    references:  https://pyqg.readthedocs.io/en/latest/index.html